In [1]:
%matplotlib inline 

In [15]:
import os
import json
import re 

import pandas as pd
import matplotlib.pyplot as plt

from pandas.io.json import json_normalize

In [16]:
def CustomParser(data):
    return json.loads(data)


FOLDER = 'results/'
regex = re.compile(r'.csv')
folders = set(os.listdir(FOLDER))
folders = filter(lambda i: regex.search(i), folders)
folders = [i for i in folders if regex.search(i)]

df = pd.DataFrame()
# Read csv file
for filename in folders:
    ddf = pd.read_csv(FOLDER + filename, sep=';', converters={'db_parameters': CustomParser, 'model_cfg': CustomParser})
    ddf[sorted(ddf['db_parameters'][0].keys())] = ddf['db_parameters'].apply(pd.Series)
    ddf[sorted(ddf['model_cfg'][0].keys())] = ddf['model_cfg'].apply(pd.Series)
    
    df = pd.concat([df, ddf])

In [17]:
df.shape

(434, 12)

In [18]:
folders = set(os.listdir(FOLDER))

regex = re.compile(r'.csv')
# use only one of the following lines, whichever you prefer
folders = filter(lambda i: not regex.search(i), folders)
folders = [i for i in folders if not regex.search(i)]

folders.remove('_sources')
folders.remove('results.tar.gz')

result = {
    'acc': [],
    'f1': [],
    'db': [],
    'task': [],
    'model': [],
    'db_parameters': []
}

for root_folder in folders:
    for run_id in os.listdir(FOLDER + root_folder):
        current_folder = FOLDER + root_folder + '/' + run_id

        metrics_file = current_folder + '/metrics-test.csv'
        config_file = current_folder + '/config.json'

        if not os.path.exists(metrics_file) or not os.path.exists(config_file):
            continue

        config_df = pd.read_json(config_file)
        if config_df['dataset_cfg']['reader'] != "pypagai.preprocessing.dataset_babi.BaBIDataset":
            continue

        db = 'babi'

        task = config_df['dataset_cfg']['task']
        model = config_df['model_default_cfg']['model'].split('.')[-1]
        
        metrics_df = pd.read_csv(metrics_file)
        acc = metrics_df['accuracy'][0]
        f1 = metrics_df['f1_micro'][0]

        result['f1'].append(f1)
        result['db'].append(db)
        result['acc'].append(acc)
        result['task'].append(task)
        result['model'].append(model)
        result['db_parameters'].append(config_file)

new_results = pd.DataFrame(result)
df = pd.concat([df, new_results])

In [25]:
# Show best result of each model for each babi task
# ddf = df[df['db'] == 'babi']
# ddf = pd.concat([new_results, ddf], keys=['acc', 'f1', 'db', 'task', 'model'])

ddf = df[df['db'] == 'babi']
ddf = ddf.groupby(['db', 'model', 'task'])
ddf = ddf.max()
ddf = ddf.reset_index()
ddf[['model', 'acc', 'task']].pivot('task', 'model')


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead tr th {
        text-align: left;
    }

    .dataframe thead tr:last-of-type th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th colspan="12" halign="left">acc</th>
    </tr>
    <tr>
      <th>model</th>
      <th>ConvInputsRN</th>
      <th>ConvQueryRN</th>
      <th>ConvRN</th>
      <th>ConvStoryRN</th>
      <th>DeepN2NMemory</th>
      <th>EmbedLSTM</th>
      <th>EncoderModel</th>
      <th>N2NMemory</th>
      <th>RN</th>
      <th>RNNModel</th>
      <th>RNNoLSTM</th>
      <th>SimpleLSTM</th>
    </tr>
    <tr>
      <th>task</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>0.354</td>
      <td>0.441</td>
      <td>0.157</td>
      <td>0.149</td>
      <td>0.963</td>
      <td>0.711</td>
      <td>0.349</td>
      <td>0.953</td>
      <td>0.367</td>
      <td>0.511</td>
      <td>0.813</td>
      <td>0.504</td>
    </tr>
    <tr>
      <th>2</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.325</td>
      <td>NaN</td>
      <td>0.370</td>
      <td>0.281</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.190</td>
    </tr>
    <tr>
      <th>3</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.251</td>
      <td>NaN</td>
      <td>0.310</td>
      <td>0.242</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.207</td>
    </tr>
    <tr>
      <th>4</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>1.000</td>
      <td>NaN</td>
      <td>0.572</td>
      <td>1.000</td>
      <td>1.000</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.279</td>
    </tr>
    <tr>
      <th>5</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.826</td>
      <td>NaN</td>
      <td>0.313</td>
      <td>0.869</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.344</td>
    </tr>
    <tr>
      <th>6</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.497</td>
      <td>0.972</td>
      <td>NaN</td>
      <td>0.527</td>
      <td>0.995</td>
      <td>0.822</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.636</td>
    </tr>
    <tr>
      <th>7</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.904</td>
      <td>NaN</td>
      <td>0.764</td>
      <td>0.879</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.693</td>
    </tr>
    <tr>
      <th>8</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.802</td>
      <td>NaN</td>
      <td>0.740</td>
      <td>0.804</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.539</td>
    </tr>
    <tr>
      <th>9</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.962</td>
      <td>NaN</td>
      <td>0.585</td>
      <td>0.959</td>
      <td>0.829</td>
      <td>NaN</td>
      <td>0.827</td>
      <td>0.655</td>
    </tr>
    <tr>
      <th>10</th>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.956</td>
      <td>NaN</td>
      <td>0.429</td>
      <td>0.981</td>
      <td>0.703</td>
      <td>NaN</td>
      <td>0.732</td>
      <td>0.538</td>
    </tr>
    <tr>
      <th>11</th>
      <td>NaN</td>
      <td>0.721</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>0.985</td>
      <td>NaN</td>
      <td>0.649</td>
      <td>0.998</td>
      <td>0.985</td>
     